# Chapter 4: Dynamic Programming

### Exercise 4.1

Q: In Example 4.1, if $\pi$ is the equiprobable random policy, what is $q_\pi(11, \texttt{down})$? What is $q_\pi(7, \texttt{down})$?

A: Example 4.1 describes a 4-by-4 gridworld where the top-left and bottom-right cells are terminal states. Each transition has reward -1 (including invalid actions) and there is no discounting. The (non-terminal) states are numbered left-to-right, then top-to-bottom, starting from 1 for state $(0, 1)$. Since there is no discounting, the (absolute) value of any state corresponds to the expected number of steps it will take to reach a terminal state from that state under the current policy.

Let us start by specifying this gridworld as code and deriving, using iterative policy evaluation, the state value function $v_\pi$ for this policy:

In [ ]:
from numpy.testing import assert_almost_equal
import numpy as np
from dp import GridWorld
from dp.solve import iterative_policy_evaluation, StateValueFunction

# Gridworld specified in Example 4.1 of Sutton, Barto (2018)
gridworld_4_1 = GridWorld(
    size=4,
    terminal_states=[(0, 0), (3, 3)],
    default_move_reward=-1.0,
    invalid_action_reward = -1.0,
)

# Evaluate equiprobably random policy
v = StateValueFunction(gridworld_4_1)
def pi(state: tuple[int, int]) -> list[tuple[str, float]]:
    return [(a, 0.25) for a in gridworld_4_1.actions(state)]
iterative_policy_evaluation(v, pi, 1.0)

# Check against the result in the textbook
desired_v = np.array([
    [0.0, -14.0, -20.0, -22.0],
    [-14.0, -18.0, -20.0, -20.0],
    [-20.0, -20.0, -18.0, -14.0],
    [-22.0, -20.0, -14.0, 0.0]
]).ravel()
assert_almost_equal(
    actual=list(v.to_dict().values()),
    desired=desired_v,
    decimal=2,
)

Once we know $v_\pi$, we can compute action values $q_\pi(s, a)$ using the relationship:

$$q_\pi(s, a) = \mathbb{E}_\pi\left[R_{t+1} + \gamma\,v_\pi(S_{t+1}) \mid S_t=s, A_t=a\right].$$

This means, by manual computation (and with $\gamma = 1$):

$$
\begin{align}
q_\pi(11, \texttt{down}) &= -1\\
q_\pi(7, \texttt{down}) &= -15.\\
\end{align}
$$

We can check these results against numerically computing these action values:

In [ ]:
assert_almost_equal(
    actual=v.backup_action_value(gridworld_4_1.i2s(11), "s", gamma=1.0),
    desired=-1.0,
    decimal=2,
)
assert_almost_equal(
    actual=v.backup_action_value(gridworld_4_1.i2s(7), "s", gamma=1.0),
    desired=-15.0,
    decimal=2,
)

### Exercise 4.2

Q: In example 4.1, suppose a new state 15 is added to the gridworld just below state 13, and its actions, $\texttt{left}$, $\texttt{up}$, $\texttt{right}$, and $\texttt{down}$, take the agent to states 12, 13, 14, and 15, respectively. Assume that the transitions *from* the original states are unchanged. What, then, is $v_\pi(15)$ for the equiprobable random policy? Now suppose the dynamics of state 13 are also changed, such that action $\texttt{down}$ from state 13 takes the agent to the new state 15. What is $v_\pi(15)$ for the equiprobable random policy in this case?

A: For the first part of the question, we assume transitions from the original states are unchanged. This means that state 15 is not reachable from any of the original states and therefore has no impact on the original states' values. Since

$$v_\pi(15) = -1 + \tfrac{v_\pi(12) + v_\pi(13) + v_\pi(14) + v_\pi(15)}{4},$$

we can rearrange and insert the original state values for states 12, 13 and 14 to find

$$v_\pi(15) = \frac{4}{3} \, \left(-1 + \tfrac{-22 -20 -14}{4}\right) = -20.$$

In the second part of the question, we suppose that going down from state 13 moves the agent to state 15 and are against asked for the value of state 15. We can tackle this by performing iterative policy evaluation, starting from $v_\pi$ as given in the first part of the question. We begin by updating the value of $v_\pi(13)$ but immediately notice that this won't change since (in the first part of the question) $v_\pi(15) = v_\pi(13)$. Therefore we see that none of the other $v_\pi(s)$ will need updating either, i.e. we have already converged. In other words, introducing this transition makes no difference to $v_\pi(15)$ for the equiprobably random policy: it is still -20.

Let's check these answers against a numerical solution. First let's define a `FiniteMDP` class to represent the modified gridworld problem in this exercise. It encapsulates a standard `GridWorld` but augments it to include custom states and transitions, plus uses the state numbering scheme
provided in the exercise:

In [ ]:
from itertools import chain
from dp import FiniteMDP

class ModifiedGW(FiniteMDP):
    """Gridworld with added states and transitions,
    as per exercise 4.2 of Sutton, Barto (2018)."""
    
    def __init__(
        self, custom_states_and_transitions):
        self._inner_gw = GridWorld(
            4,
            terminal_states=((0, 0), (3, 3)),
            default_move_reward=-1.0,
            invalid_action_reward=-1.0,
        )
        self.custom_states_and_transitions = custom_states_and_transitions
    
    def mgw2gw_state(self, mgw_state):
        """Translates states (1 to 15) in the exercise to original
        GridWorld states (tuples of ints), where possible. For custom
        states not in the original gridworld, returns None."""
        if mgw_state in self.custom_states_and_transitions.keys():
            return None
        else:
            return self._inner_gw.i2s(mgw_state)
    
    def gw2mgw_state(self, gw_state):
        """Translates original GridWorld states (tuples of ints) to
        the states numbered as per the exercise. Note, 0 represents
        both the terminal states."""
        if gw_state in self._inner_gw.terminal_states:
            return 0
        else:
            return self._inner_gw.s2i(gw_state)
    
    @property
    def states(self):
        return sorted(
            set(range(0, 15)) | self.custom_states_and_transitions.keys()
        )
    
    @property
    def actions(self, state):
        return ("n", "e", "w", "s")
    
    def next_states_and_rewards(self, state: int, action: str):
        if state in self.custom_states_and_transitions.keys():
            next_state = self.custom_states_and_transitions[state][action]
            reward = -1
        else:
            (nss, ps), reward = self._inner_gw.next_states_and_rewards(
                self.mgw2gw_state(state), action
            )
            assert len(nss) == 1
            assert ps[0] == 1.0
            next_state = self.gw2mgw_state(nss[0])
        return ((next_state,), (1.0,),), reward

Now we answer the first part of the question, numerically, by adding a custom state 15, but keeping all other state transitions as they were before.
We then check that an equiprobably random policy yields exactly the same $v_\pi$ as the original problem for original states, as well as a value
of -20 for state 15:

In [ ]:
# Set up modified gridworld and run iterative policy evaluation
v_modified_1 = StateValueFunction(
    ModifiedGW(
        {15: {"w": 12, "n": 13, "e": 14, "s": 15}}
    )
)
iterative_policy_evaluation(v_modified_1, pi, gamma=1.0)

# Check if state values are as expected
for s in range(0, 14):
    assert_almost_equal(v_modified_1(s), desired_v[s], decimal=2)
assert_almost_equal(v_modified_1(15), -20.0, decimal=2)

Now we do the same thing for the second part of the question, this time adding custom transitions to state 13 as well as a new state 15. Again we should find the same state value function:

In [ ]:
v_modified_2 = StateValueFunction(
    ModifiedGW(
        {
            13: {"w": 12, "n": 9, "e": 14, "s": 15},
            15: {"w": 12, "n": 13, "e": 14, "s": 15},
        },
    )
)
iterative_policy_evaluation(v_modified_2, pi, gamma=1.0)

for s in range(0, 16):
    assert_almost_equal(v_modified_2(s), v_modified_1(s), decimal=2)

### Exercise 4.3

Q: What are the equations analogous to (4.3), (4.4), and (4.5) for the action-value function $q_\pi$ and its successive approximation by a sequence of functions $q_0$, $q_1$, $q_2$, ...?

A: The equations referenced in the question develop the iterative policy evaluation algorithm for approximating the state-value function $v_\pi$. To do the same for the action-value function $q_\pi$, we begin with the Bellman equation for the action-value function:

$$
\begin{align}
q_\pi(s, a) &\doteq \mathbb{E}_\pi \left[ R_{t+1} + \gamma \, q_\pi(S_{t+1}, A_{t+1}) \mid S_t=s, A_t=a\right] \\
&= \sum_{s' \in \mathcal{S}} \sum_{a' \in \mathcal{A}} \pi(a' | s') \, p(s' | s, a) \left[r(s, a, s') + \gamma\,q_\pi(s', a') \right]
\end{align}
$$

This equation is satisfied by a fixed point of the following iteration, leading directly to a iterative policy evaluation algorithm that can be applied directly to action-values:

$$
q_{i+1}(s, a) \doteq \sum_{s' \in \mathcal{S}} \sum_{a' \in \mathcal{A}} \pi(a' | s') \, p(s' | s, a) \left[r(s, a, s') + \gamma\,q_i(s', a') \right]
$$

### Exercise 4.4

Q: The policy iteration algorithm on page 80 has a subtle bug in that it may never terminate if the policy continually switches between two or more policies that are equally good. This is ok for pedagogy, but not for actual use. Modify the pseudocode so that convergence is guaranteed.

A: The scenario described in the question can occur if there are states where two or more actions have the same action value. If different such actions can be selected for different iterations then *policy-stable* can end up remaining false even though the algorithm has actually converged. To fix this, we can update the *if* condition for determining whether *policy-stable* should be set to false. In the pseudocode, *policy-stable* is set to false whenever *old-action* differs from $\pi(s)$; instead the condition should be whether the action-value corresponding to $old-action$ differs from the action-value for $\pi(s)$. In practice (and in the implementation in the `dp` package), due to numerical error, we should check whether the action-values for the old and new policy actions are close enough (within some tolerance); where this is the case, we should not set *policy-stable* to false, as that iteration of the update has not resulted in a better policy (within tolerance).

### Exercise 4.5

Q: How would policy iteration be defined for action values? Give a complete algorithm for computing $q_*$, analogous to that on page 80 for computing $v_*$. Please pay special attention to this exercise, because the ideas involved will be used throughout the rest of the book.